In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [80]:
import fastai
fastai.__version__ 
#check fast ai version

'2.7.14'

In [81]:
from fastai import *
from fastai.tabular.all import *

pd.options.display.max_rows = 20
pd.options.display.max_columns = None

In [82]:
path1 =Path('/kaggle/input/titanic/train.csv')
path2 =Path('/kaggle/input/titanic-extended')

train_data = pd.read_csv("/kaggle/input/titanic/train.csv" , low_memory=False)
train_data.head()
# train data has the data where we have the results of who survived the data which we will use the infomation from this data to predict who will survive froom the test data if they were on the titantic
# we obtain the data from kaggle and verify that we have data by checking some of the entries

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Create TabularPandas**

In [83]:
procs = [Categorify, FillMissing] # RF does not need `Normalize`


In [84]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_data))


In [85]:
dep_var = 'Survived'

In [86]:
cont,cat = cont_cat_split(train_data, 1, dep_var=dep_var)

In [87]:
cont

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [88]:
cat

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [89]:
to = TabularPandas(train_data, procs, cat, cont, y_names=dep_var, splits=splits, y_block=CategoryBlock())

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [90]:
to.show(3)

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
869,"Johnson, Master. Harold Theodor",male,347742,#na#,S,False,870,3,4.0,1,1,11.1333,1
596,"Leitch, Miss. Jessie Wills",female,248727,#na#,S,True,597,2,28.0,0,0,33.0000,1
866,"Duran y More, Miss. Asuncion",female,SC/PARIS 2149,#na#,C,False,867,2,27.0,1,0,13.8583,1


In [91]:
# test data has the data we want to predict who will survive 
test_data = pd.read_csv("/kaggle/input/titanic/test.csv", low_memory=False)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [92]:
# we go through the data set to find the rate of female surviors 
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)


% of women who survived: 0.7420382165605095


In [93]:
# we go through the data set to find the rate of male surviors 
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


# **Create Training Set and Validation Set**
# 

In [94]:
X_train, y_train = to.train.xs,to.train.y
X_valid, y_valid = to.valid.xs,to.valid.y

In [95]:
# from sklearn.ensemble import RandomForestClassifier
# # we are using a  random forest model to run the data
# y = train_data["Survived"]

# features = ["Pclass", "Sex", "SibSp", "Parch"]
# X = pd.get_dummies(train_data[features])
# X_test = pd.get_dummies(test_data[features])

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X, y)
# predictions = model.predict(X_test)

# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submission.csv', index=False)

# print("Your submission was successfully saved!")

In [96]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, n_jobs=-1)

In [97]:
model.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1)

In [98]:
from sklearn.metrics import accuracy_score

In [99]:
y_pred=model.predict(X_valid)


In [100]:
accuracy_score(y_valid, y_pred)


0.8426966292134831

In [101]:
to_test = TabularPandas(test_data, procs, cat, cont)

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [102]:
predicted_result = model.predict(to_test.xs.drop('Fare_na', axis=1)) # remove reduntant columns (training did not use this col)

In [104]:
output= pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived': predicted_result.astype(int)})
output.to_csv('submission.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
